# How much does the image position vary due to hexapod motions?

Craig Lage  19-Aug-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import polynomial as npp
import matplotlib as mpl
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData, calcNextDay, getMostRecentRowWithDataBefore
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from astropy.time import Time, TimeDelta
import yaml

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])

In [ ]:
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/laser_rotation_elevation_09sep25.yaml"
lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init_11sep25.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)


In [ ]:
names = ['X', 'Y', 'Z', 'U', 'V']
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.8, hspace=0.6)
plt.suptitle(f"LUTs", fontsize=18, y=1.0)
els = np.linspace(20, 85, 14)
norm = mpl.colors.Normalize(vmin=-75, vmax=75)

for i in range(5):
    for r in [-45, 0, 45]:
        rots = np.linspace(r, r, 14)
        lut_coeffs = np.array(lut_data['camera_config']['elevation_rotation_coeffs'][i])
        lut = npp.polyval2d(els, rots, lut_coeffs)
        if i in [3, 4]:
            lut *= 3600.0
        axs[0][i].scatter(els, lut, marker='x', label=f"Rot={r:.0f}")
        axs[0][i].set_title(f'Cam{names[i]}')
        if i < 3:
            axs[0][i].set_ylabel("Microns")
        else:
            axs[0][i].set_ylabel("Arcseconds")
        lut_coeffs = np.array(lut_data['m2_config']['elevation_rotation_coeffs'][i])
        lut = npp.polyval2d(els, rots, lut_coeffs)
        if i in [3, 4]:
            lut *= 3600.0
        axs[1][i].scatter(els, lut, marker='x', label=f"Rot={r:.0f}")
        axs[1][i].set_title(f'M2{names[i]}')
        if i < 3:
            axs[1][i].set_ylabel("Microns")
        else:
            axs[1][i].set_ylabel("Arcseconds")
        if i == 3:
            axs[1][i].legend()
#plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Hexapod_Changes_LUT_{startDay}.png")
#plt.clf


In [ ]:
great = np.loadtxt("/home/c/cslage/u/Hexapods/data/Great_Images_20250501-20250728.txt", skiprows=1)
expIds = []
greatRots = []
greatEls = []
camPoss = []
m2Poss = []

for n in range(len(great)):
    expId =  int(great[n][0])  
    dataId = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    (mountErrors, mountData) = calculateMountErrors(expRecord, client)
    rotVals = mountData.rotationData['actualPosition'].values
    rot = (np.median(rotVals))
    greatRots.append(rot)
    elVals = mountData.elevationData['actualPosition'].values
    el = (np.median(elVals))
    greatEls.append(el)
    poss = []
    for i in range(5):
        val = np.median(mountData.camhexData[f"position{i}"].values)
        if i in [3, 4]:
            val *= 3600.0
        poss.append(val)
    camPoss.append(poss)
    poss = []
    for i in range(5):
        val = np.median(mountData.m2hexData[f"position{i}"].values)
        if i in [3, 4]:
            val *= 3600.0
        poss.append(val)
    m2Poss.append(poss)
    expIds.append(expId)
    print(f"{expId} succeeded!")


In [ ]:
names = ['X', 'Y', 'Z', 'U', 'V']
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.6, hspace=0.5)
plt.suptitle(f"LUT position for best images (FWHM < 0.7)", fontsize=18, y=0.98)
els = np.linspace(20, 85, 14)

cmap = mpl.cm.cool
norm = mpl.colors.Normalize(vmin=-75, vmax=75)
rot_plots = [-75, 0, 75]
for i in range(5):
    for j, r in enumerate(rot_plots):
        rots = np.linspace(r, r, 14)
        lut_coeffs = np.array(lut_data['camera_config']['elevation_rotation_coeffs'][i])
        lut = npp.polyval2d(els, rots, lut_coeffs)
        if i in [3, 4]:
            lut *= 3600.0
        axs[0][i].scatter(els, lut, marker='x', label=f"Rot={r:.0f}", c=rots, vmin=-75, vmax=75, cmap='viridis')
        cam = [term[i] for term in camPoss]
        #print(len(greatEls), len(cam))
        axs[0][i].scatter(greatEls, cam, marker='.', c=greatRots, vmin=-75, vmax=75, cmap='viridis')
        if j == 2:
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='viridis'),
                 ax=axs[0][i], orientation='vertical')
            cbar.set_label('Rot', labelpad=-10)
        axs[0][i].set_title(f'Cam{names[i]}')
        if i < 3:
            axs[0][i].set_ylabel("Microns")
        else:
            axs[0][i].set_ylabel("Arcseconds")
        axs[0][i].set_xlabel("Elevation")
        lut_coeffs = np.array(lut_data['m2_config']['elevation_rotation_coeffs'][i])
        lut = npp.polyval2d(els, rots, lut_coeffs)
        if i in [3, 4]:
            lut *= 3600.0
        axs[1][i].scatter(els, lut, marker='x', label=f"Rot={r:.0f}", c=rots, vmin=-75, vmax=75, cmap='viridis')
        m2 = [term[i] for term in m2Poss]
        #print(len(greatEls), len(m2))
        axs[1][i].scatter(greatEls, m2, marker='.', c=greatRots, vmin=-75, vmax=75, cmap='viridis')
        if j == 2:
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='viridis'),
                 ax=axs[1][i], orientation='vertical')
            cbar.set_label('Rot', labelpad=-10)
        axs[1][i].set_title(f'M2{names[i]}')
        if i < 3:
            axs[1][i].set_ylabel("Microns")
        else:
            axs[1][i].set_ylabel("Arcseconds")
        axs[1][i].set_xlabel("Elevation")
        if i == 4:
            axs[1][i].legend(fontsize=8, loc='upper left')
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Hexapods_Great_Images_3.png")
#plt.clf


In [ ]:
expIds = []
greatRots = []
greatEls = []
camPoss = []
m2Poss = []
seqNums = []
start = 224
end = 251
dayObs = 20250911
for seqNum in range(start, end+1):
    expId = int(dayObs * 1E5 + seqNum)
    dataId = {'exposure':expId, 'instrument':'LSSTCam'}
    expRecord = getExpRecordFromDataId(butler, dataId)
    (mountErrors, mountData) = calculateMountErrors(expRecord, client)
    rotVals = mountData.rotationData['actualPosition'].values
    rot = (np.median(rotVals))
    greatRots.append(rot)
    elVals = mountData.elevationData['actualPosition'].values
    el = (np.median(elVals))
    greatEls.append(el)
    poss = []
    for i in range(5):
        val = np.median(mountData.camhexData[f"position{i}"].values)
        if i in [3, 4]:
            val *= 3600.0
        poss.append(val)
    camPoss.append(poss)
    poss = []
    for i in range(5):
        val = np.median(mountData.m2hexData[f"position{i}"].values)
        if i in [3, 4]:
            val *= 3600.0
        poss.append(val)
    m2Poss.append(poss)
    expIds.append(expId)
    seqNums.append(seqNum)
    print(f"{expId} succeeded!")


In [ ]:
names = ['X', 'Y', 'Z', 'U', 'V']
fig, axs = plt.subplots(2, 5, figsize=(16, 5))
plt.subplots_adjust(wspace=0.7, hspace=0.5)
plt.suptitle(f"LUT position for hexapod closed loop - seqNums {start}-{end}", fontsize=18, y=0.98)
els = np.linspace(20, 85, 14)

cmap = mpl.cm.cool
norm = mpl.colors.Normalize(vmin=start, vmax=end)
rot_plots = [0]
for i in range(5):
    for j, r in enumerate(rot_plots):
        rots = np.linspace(r, r, 14)
        lut_coeffs = np.array(lut_data['camera_config']['elevation_rotation_coeffs'][i])
        lut = npp.polyval2d(els, rots, lut_coeffs)
        if i in [3, 4]:
            lut *= 3600.0
        axs[0][i].scatter(els, lut, marker='x', label=f"Rot={r:.0f}")
        cam = [term[i] for term in camPoss]
        axs[0][i].scatter(greatEls, cam, marker='o', c=seqNums, norm=norm, cmap='Spectral')
        if j == 0:
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='Spectral'),
                 ax=axs[0][i], orientation='vertical')
            cbar.set_label('seqNum', labelpad=0)
        axs[0][i].set_title(f'Cam{names[i]}')
        if i < 3:
            axs[0][i].set_ylabel("Microns")
        else:
            axs[0][i].set_ylabel("Arcseconds")
        axs[0][i].set_xlabel("Elevation")
        lut_coeffs = np.array(lut_data['m2_config']['elevation_rotation_coeffs'][i])
        lut = npp.polyval2d(els, rots, lut_coeffs)
        if i in [3, 4]:
            lut *= 3600.0
        axs[1][i].scatter(els, lut, marker='x', label=f"Rot={r:.0f}")
        m2 = [term[i] for term in m2Poss]
        axs[1][i].scatter(greatEls, m2, marker='o',c=seqNums, norm=norm, cmap='Spectral')
        if j == 0:
            cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap='Spectral'),
                 ax=axs[1][i], orientation='vertical')
            cbar.set_label('SeqNum', labelpad=0)
        axs[1][i].set_title(f'M2{names[i]}')
        if i < 3:
            axs[1][i].set_ylabel("Microns")
        else:
            axs[1][i].set_ylabel("Arcseconds")
        axs[1][i].set_xlabel("Elevation")
        if i == 4:
            axs[1][i].legend(fontsize=8, loc='center left')
plt.savefig(f"/home/c/cslage/u/Hexapods/LUTs/Hexapods_Closed_Loop_20250911_{start}-{end}.png")
#plt.clf


In [ ]:
seqNums